In [ ]:
!pip install -q dwave-ocean-sdk

In [ ]:
!pip install cheche_pm

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import product
import time
from tqdm import tqdm
from cheche_pm import Project

In [ ]:
import dimod
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import dwave.inspector as inspector
from dwave.system.samplers import DWaveSampler
from dimod.binary.binary_quadratic_model import BinaryQuadraticModel
from dimod.vartypes import Vartype

In [ ]:
endpoint = 'https://cloud.dwavesys.com/sapi'
token = '<ENTER YOUR DWAVE API KEY HERE>'

# Input data

In [ ]:
inst = '< Insert the instance name here>'

In [ ]:
project = Project.from_rangen_1_rcp_file(file_name = inst)

In [ ]:
data = project.produce_outputs_for_MILP()

In [ ]:
n,p,S,u,c,ph = tuple(data.values())

(R, J, T) = (range(len(c)), range(len(p)), range(ph))
(R, J, T)
M = sum(p)

In [ ]:
nbOfActivities = n
nbOfPeriods = M
nbOfRessources = len(c)
timeActivities = p
capaRessources = c
activityRessourceConsumption = u
pairsListPrecedence = S
annealsNumber = 1000

In [ ]:
lambda_equalityConst =  nbOfPeriods
lambda_precedenceConst =  nbOfPeriods
lambda_ressourceConst =  nbOfPeriods
bigInteger  = 999999

# Create the BQM

In [ ]:
bqm = BinaryQuadraticModel(Vartype.BINARY)

In [ ]:
x = [[f'x_{i}_{t}' for t in T] for i in J]

In [ ]:
for t in T:
  bqm.add_variable(x[nbOfActivities+1][t], t)

## Objective function

In [ ]:
for t in T:
  bqm.add_variable(x[nbOfActivities+1][t], t)

## Constraints

### C1: Each activity must have an start date


In [ ]:
for i in J:
  terms = x[i]
  terms = [(x,1) for x in terms]
  bqm.add_linear_equality_constraint(terms=terms,lagrange_multiplier=lambda_equalityConst,constant=-1)
  terms.clear

### C2: Precedence constraints

In [ ]:
for precedenceIndex in range(len(pairsListPrecedence)): # precedence constraints
    activityOne = pairsListPrecedence[precedenceIndex][0]
    activityTwo = pairsListPrecedence[precedenceIndex][1]
    timeActivityOne = timeActivities[activityOne]
    for p1 in range(nbOfPeriods):
        for p2 in range(nbOfPeriods):
            if (p1 + timeActivityOne > p2):
                nameVar1 = "x_" + str(activityOne) +  "_" + str(p1)
                nameVar2 = "x_" + str(activityTwo) +  "_" + str(p2)
                bqm.add_quadratic(nameVar1,nameVar2,lambda_precedenceConst) # venturelli constraint
                #print("x ", nameVar1, " * ", nameVar2, " = 0 ")

### C3: Resource constraints

In [ ]:
for r in R:
  for t in T:
    terms = [(f'x_{j}_{t2}',u[j][r]) for j in J for t2 in range(max(0, t - p[j] + 1), t + 1)]
    bqm.add_linear_inequality_constraint(terms=terms,
                                         lagrange_multiplier=lambda_ressourceConst,
                                         label='res_',
                                         constant = -c[r],
                                         lb = -bigInteger,
                                         ub = 0)
    terms.clear

In [ ]:
print(dict(bqm.linear))

print("\n")

print(dict(bqm.quadratic))

In [ ]:
len(dict(bqm.linear))

In [ ]:
bqm.to_qubo() # genera un diccionario

# Define annealing schedule

In [ ]:
def make_reverse_anneal_schedule(s_target=0.0, hold_time=10.0, ramp_back_slope=0.2, ramp_up_time=0.0201,
                                 ramp_up_slope=None):
    """Build annealing waveform pattern for reverse anneal feature.

    Waveform starts and ends at s=1.0, descending to a constant value
    s_target in between, following a linear ramp.

      s_target:   s-parameter to descend to (between 0 and 1)
      hold_time:  amount of time (in us) to spend at s_target (must be >= 2.0us)
      ramp_slope: slope of transition region, in units 1/us
    """
    # validate parameters
    if s_target < 0.0 or s_target > 1.0:
        raise ValueError("s_target must be between 0 and 1")
    if hold_time < 0.0:
        raise ValueError("hold_time must be >= 0")
    if ramp_back_slope > 0.2:
        raise ValueError("ramp_back_slope must be <= 0.2")
    if ramp_back_slope <= 0.0:
        raise ValueError("ramp_back_slope must be > 0")

    ramp_time = (1.0 - s_target) / ramp_back_slope

    initial_s = 1.0
    pattern = [[0.0, initial_s]]

    # don't add new points if s_target == 1.0
    if s_target < 1.0:
        pattern.append([round(ramp_time, 4), round(s_target, 4)])
        if hold_time != 0:
            pattern.append([round(ramp_time+hold_time, 4), round(s_target, 4)])

    # add last point
    if ramp_up_slope is not None:
        ramp_up_time = (1.0-s_target)/ramp_up_slope
        pattern.append([round(ramp_time + hold_time + ramp_up_time, 4), round(1.0, 4)])
    else:
        pattern.append([round(ramp_time + hold_time + ramp_up_time, 4), round(1.0, 4)])

    return pattern

In [ ]:
sampler = DWaveSampler(solver='Advantage_system6.3',endpoint= endpoint, token = token)

In [ ]:
print("Connected to sampler", sampler.solver.name)

In [ ]:
print("Maximum anneal-schedule points: {}".format(sampler.properties["max_anneal_schedule_points"]))
print("Annealing time range: {}".format(sampler.properties["annealing_time_range"]))

max_slope = 1.0/sampler.properties["annealing_time_range"][0]

print("Maximum slope allowed on this solver is {:.2f}.".format(max_slope))

In [ ]:
reverse_schedule = make_reverse_anneal_schedule(s_target=0.45, hold_time=80, ramp_up_slope = max_slope)
time_total = reverse_schedule[3][0]

print(reverse_schedule)
print("Total anneal-schedule time is {} us".format(time_total))

In [ ]:
plt.figure(1, figsize=(3, 3))
plt.plot(*np.array(reverse_schedule).T)
plt.title("Reverse Anneal Schedule")
plt.xlabel("Time [us]")
plt.ylabel("Annealing Parameter s")
plt.ylim([0.0,1.0])
plt.show()

# Forward solution

In [ ]:
sampler = EmbeddingComposite(sampler)
sampler_name = sampler.properties['child_properties']['chip_id']

In [ ]:
forward_answer = sampler.sample(bqm, num_reads= 1000)

In [ ]:
forward_solutions, forward_energies = forward_answer.record.sample, forward_answer.record.energy

In [ ]:
best = np.argmin(forward_energies)

In [ ]:
initial = dict(zip(forward_answer.variables, forward_answer.record[best].sample))

print("Lowest energy found: {}".format(forward_answer.record.energy[best]))
print("Average energy is {:.2f} with standard deviation {:.2f}".format(forward_energies.mean(), forward_energies.std()))
print("\nSetting the initial state to a sample with energy: {}".format(forward_answer.record.energy[best]))

# Reverse Annealing

In [ ]:
reverse_anneal_params = dict(anneal_schedule=reverse_schedule, initial_state=initial, reinitialize_state=True)

In [ ]:
reverse_answer = sampler.sample(bqm, num_reads= 1000,**reverse_anneal_params)

In [ ]:
reverse_solutions, reverse_energies = reverse_answer.record.sample, reverse_answer.record.energy

reverse_best = np.argmin(reverse_energies)

print("Lowest energy found:", reverse_answer.record.energy[reverse_best])
print("Average energy is {:.2f} with standard deviation {:.2f}".format(reverse_energies.mean(), reverse_energies.std()))

In [ ]:
reverse_best